In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [4]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 7178 images belonging to 7 classes.


In [5]:
# create model structure
emotion_model = Sequential()

In [6]:
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [7]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [8]:
cv2.ocl.setUseOpenCL(False)

In [9]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

C:\Users\ankit\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

<ipython-input-10-ae419f95bfec>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 506s 1s/step - loss: 1.8035 - accuracy: 0.2588 - val_loss: 1.7288 - val_accuracy: 0.3098
Epoch 2/50
448/448 [==============================] - 481s 1s/step - loss: 1.6467 - accuracy: 0.3571 - val_loss: 1.5711 - val_accuracy: 0.4082
Epoch 3/50
448/448 [==============================] - 356s 794ms/step - loss: 1.5490 - accuracy: 0.4041 - val_loss: 1.4868 - val_accuracy: 0.4382
Epoch 4/50
448/448 [==============================] - 384s 857ms/step - loss: 1.4766 - accuracy: 0.4332 - val_loss: 1.4204 - val_accuracy: 0.4570
Epoch 5/50
448/448 [==============================] - 490s 1s/step - loss: 1.4191 - accuracy: 0.4600 - val_loss: 1.3794 - val_accuracy: 0.4760
Epoch 6/50
448/448 [==============================] - 430s 959ms/step - loss: 1.3670 - accuracy: 0.4794 - val_loss: 1.3379 - val_accuracy: 0.4954
Epoch 7/50
448/448 [==============================] - 375s 837ms/step - loss: 1.3236 - accuracy: 0.4985 - val_loss: 1.3043 - val_accu

In [11]:
# save model structure in json file
model_json = emotion_model.to_json()
with open("emotion_model50.json", "w") as json_file:
    json_file.write(model_json)

In [12]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model50.h5')